In [19]:
import matplotlib.pyplot as plt
import json
import glob
import os
import numpy as np
from config import problem_configs
from helpers import calculate_ising_energy


In [20]:
#///////// Config //////////
FILEDIRECTORY = "isingBatches"

In [21]:
def load_file_into_dict(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

In [22]:
for problem_name, config in problem_configs.items():
        print(f"--- Processing class: {problem_name} ---")

        file_slug = config['file_slug']
        search_pattern = os.path.join(FILEDIRECTORY, f"MERGED*{file_slug}.json")
        distributionFile = glob.glob(search_pattern)[0]
        search_pattern = os.path.join(FILEDIRECTORY, f"solved*{file_slug}.json")
        exactSolutionsFile = glob.glob(search_pattern)[0]
        search_pattern = os.path.join(FILEDIRECTORY, f"batch*{file_slug}.json")
        isingModelsFile = glob.glob(search_pattern)[0]


        print(f'Distribution file = {distributionFile}, Solutions file = {exactSolutionsFile}, Ising models file = {isingModelsFile}')

        sampledResultsData = load_file_into_dict(distributionFile)
        exactSolutions = load_file_into_dict(exactSolutionsFile)
        isingModels = load_file_into_dict(isingModelsFile)

        qaoaSuccessCount = 0

        normalisedPerformanceScores = []
        for run_instance in sampledResultsData["results"]:
                instance_id = run_instance["instance_id"]

                # The 'sampled_distribution' is now a sorted list of lists.
                # The most probable solution is the first item in the list.
                # e.g., ["010100001", 39]. We only need the bitstring, which is at index [0].
                most_probable_bitstring = run_instance["sampled_distribution"][0][0]

                # The original script reversed the string, so we preserve that logic.
                mostProbableSolution = most_probable_bitstring[::-1]

                # Find the corresponding exact solutions for this instance
                correctSolutionsIsing = []
                for exact_item in exactSolutions:
                        # Compare the integer instance_id directly
                        if exact_item["instance_id"] == instance_id:
                                correctSolutionsIsing = exact_item["solutions"]
                                optimumCost = exact_item["cost"]
                                maximumCost = exact_item["max_cost"]
                                # Optimization: once found, no need to continue searching
                                break 
                
                # This logic for converting from Ising to binary strings remains the same
                correctSolutionsBinary = []
                for ising_string in correctSolutionsIsing:
                        # 0s and 1s may seem mixed up here, but it's because the qubit state |0> 
                        # corresponds to the Z-eigenvalue of +1, and vice-versa.
                        binary_string = ising_string.replace('-1', '1').replace('+1', '0').replace(',', '')
                        correctSolutionsBinary.append(binary_string)
                # Check if the QAOA result is one of the valid ground states
                if mostProbableSolution in correctSolutionsBinary:
                        qaoaSuccessCount += 1

                for isingModel in isingModels:
                        if isingModel["instance_id"] == instance_id:
                                isingTerms = isingModel["terms"]
                                isingWeights = isingModel["weights"]
                                mostProbableSolutionCost = calculate_ising_energy(mostProbableSolution, isingTerms, isingWeights)
                                energyRange = maximumCost - optimumCost
                                performance = (mostProbableSolutionCost - optimumCost) / energyRange if energyRange != 0 else 0
                                normalisedPerformanceScores.append(performance)
                                break
        averagePerformance = np.mean(normalisedPerformanceScores)
 
        # Print the final total success count
        print(f"Total QAOA Success Count: {qaoaSuccessCount} / {len(sampledResultsData['results'])}")
        print(f"Average Normalised Performance Score: {averagePerformance:.4f}")

--- Processing class: TSP ---
Distribution file = isingBatches/MERGED_QAOA_results_TSP_9q_.json, Solutions file = isingBatches/solved_batch_Ising_data_TSP_9q_.json, Ising models file = isingBatches/batch_Ising_data_TSP_9q_.json
Total QAOA Success Count: 35 / 100
Average Normalised Performance Score: 0.0172
--- Processing class: Knapsack ---
Distribution file = isingBatches/MERGED_QAOA_results_Knapsack_6_items_9q_.json, Solutions file = isingBatches/solved_batch_Ising_data_Knapsack_6_items_9q_.json, Ising models file = isingBatches/batch_Ising_data_Knapsack_6_items_9q_.json
Total QAOA Success Count: 3 / 100
Average Normalised Performance Score: 0.0115
--- Processing class: MinimumVertexCover ---
Distribution file = isingBatches/MERGED_QAOA_results_MinimumVertexCover_9q_.json, Solutions file = isingBatches/solved_batch_Ising_data_MinimumVertexCover_9q_.json, Ising models file = isingBatches/batch_Ising_data_MinimumVertexCover_9q_.json
Total QAOA Success Count: 4 / 100
Average Normalised 